# Laden der OR Tools

*(nur notwendig, wenn noch nicht installiert)*

Link zum Notebook in Colab Research: [Link](https://colab.research.google.com/drive/1sjqX_7IK_bb-_CyTytvugFoYZn7B11Kh?usp=sharing)

In [ ]:
!pip install ortools

# Solver initialisieren & Parameterdefinition

* Lädt notwendige Bibliotheken und initialisiert Solver

* Definiert Parameter für die folgende Instanz

![Instanz](https://drive.google.com/file/d/1JmtMs7MXilPrWLgAIai__fH3sPsTdKmO/)

In [7]:
from ortools.linear_solver import pywraplp

# Initialisiere und definiere den Solver
solver = pywraplp.Solver(
    'MIP Standort kurz', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# PARAMETERDEFINITION

# t_i Dauer der Arbeitsschritte
t = [3, 3, 4, 3, 2, 7, 5, 3]
C = 7
I = len(t)
M = len(t)

#Nachfolger S_i
S = [[1,2,3,4,5,6,7], # 0
     [3,5,6,7],       # 1
     [4,5,7],         # 2
     [5,6,7],         # 3
     [7],             # 4
     [7],             # 5
     [7],             # 6
     []]              # 7

# Variablendeklaration

In [8]:
# VARIABLENDEKLARATION
# x_i,m aus {0,1}
x = [[]]*I
for i in range(I):
    x[i] = [solver.BoolVar("x_%s_%d" % (i, m)) for m in range(M)]

# y_m als Boolean
y = [solver.BoolVar("y_%s" % m) for m in range(M)]

# Definition der Nebenbedingungen und Zielfunktion

In [9]:
#### DEFINITION DER NEBENBEDINGUNGEN ####

# (2) Jeder AS muss zugeordnet werden (Summe x_im==1)
for i in range(I):
    sum_xim = solver.Sum([x[i][m] for m in range(M)])
    solver.Add(sum_xim == 1)

# (3) Taktzeit darf nicht überschritten werden t_i * x_im <= C*ym
for m in range(M):
    sum_ti_xim = solver.Sum([t[i]*x[i][m] for i in range(I)])
    solver.Add(sum_ti_xim <= C*y[m])

# (4) Kein Vorgänger zu späterer Station sum m*x_im <= m*x_jm
for i in range(I):
    sum_m_xim = solver.Sum([m * x[i][m] for m in range(M)])
    for j in S[i]:
      sum_m_xjm = solver.Sum([m * x[j][m] for m in range(M)])
      solver.Add(sum_m_xim <= sum_m_xjm)


# DEFINITION DER ZIELFUNKTION
nr_stations = solver.Sum(y[m] for m in range(M))
solver.Minimize(nr_stations)

# Starten des Lösungsvorgangs

inkl. Ausgabe der Lösung

In [ ]:
# TRIGGERN DES LÖSUNGSVORGANGS
status = solver.Solve()

# AUSGABE DER LÖSUNG
print('Anzahl Variablen =', solver.NumVariables())
print('Anzahl Nebenbedingungen =', solver.NumConstraints())

if status == pywraplp.Solver.OPTIMAL:
    print('Lösung:')
    print('Zielfunktionswert =', solver.Objective().Value())
    print("Stationen = ", [y[m].solution_value() for m in range(M)])
    for m in range(M):
      if y[m].solution_value()> 0:
        print("AS an Station ", m , ": ", [i for i in range(I) if x[i][m].solution_value() > 0])
        

else:
    print('Keine Lösung gefunden')

# AUSGABE DER LÖSUNGSZEIT UND ANGABEN ZUM LÖSUNGSVORGANG
print('\nLösungsvorgang:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())